In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
x1 = pd.ExcelFile('新竹海淡_交通_2nd.xlsx')
sheet_names = x1.sheet_names

In [5]:
#sheet_names
#可列出所有sheet names

In [6]:
#df1_woday = x1.parse('1.0622woday')
#df1_hoday = x1.parse('1.0621hoday')
#df2_woday = x1.parse('2.0622woday')
#df2_hoday = x1.parse('2.0621hoday')
#df3_woday = x1.parse('3.0622woday')
#df3_hoday = x1.parse('3.0621hoday')

In [7]:
df1_woday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=0)
df1_hoday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=1)
df2_woday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=2)
df2_hoday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=3)
df3_woday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=4)
df3_hoday = pd.read_excel('新竹海淡_交通_2nd.xlsx', sheet_name=5)

In [8]:
#df1_woday.head()
#測試OK

In [9]:
#df1_hoday.head()
#測試OK

In [10]:
coe_of_bike = 0.5
coe_of_vehicle = 1
coe_of_truck = 2
coe_of_special = 3

## 小型車當量(權重)

In [11]:
def Ngyi_cal(g, 車道寬度):
    if (g<= 65 and g > 15):
        Ngyi = ( -1.23 + 0.457*g + 7.352*(10**(-4))*(g**2)) * (0.94 + 0.02*車道寬度)
    else:
        Ngyi = (-4.09 + 0.549*g) * (0.94 + 0.02*車道寬度)
    return Ngyi

## 計算Ngyi(紓解車數)值
## for郊區多車道-號誌化路口

In [12]:
def Esb_cal(g):
    if g <= 85:
        Esb = 1.77 + 0.011*g -6.25 * (10**(-5)) * (g**2)
    else:
        Esb = 2.23
    return Esb
## 計算Esb(大車之小車當量)
## for郊區多車道-號誌化路口

In [13]:
def depart_direction_1stpoint(df):
    df['海濱路_往北_機踏車'] = df['直行-機1'] + df['右轉-機2'] + df['左轉-機4']
    df['海濱路_往南_機踏車'] = df['左轉-機2'] + df['直行-機3'] + df['右轉-機4']
    df['天府路_往東_機踏車'] = df['右轉-機1'] + df['左轉-機3'] + df['直行-機4']
    df['天府路_往西_機踏車'] = df['左轉-機1'] + df['直行-機2'] + df['右轉-機3']
      
    df['海濱路_往北_小型車'] = df['直行-小1'] + df['右轉-小2'] + df['左轉-小4']
    df['海濱路_往南_小型車'] = df['左轉-小2'] + df['直行-小3'] + df['右轉-小4']    
    df['天府路_往東_小型車'] = df['右轉-小1'] + df['左轉-小3'] + df['直行-小4']
    df['天府路_往西_小型車'] = df['左轉-小1'] + df['直行-小2'] + df['右轉-小3']  
      
    df['海濱路_往北_大型車'] = df['直行-大1'] + df['右轉-大2'] + df['左轉-大4']
    df['海濱路_往南_大型車'] = df['左轉-大2'] + df['直行-大3'] + df['右轉-大4']    
    df['天府路_往東_大型車'] = df['右轉-大1'] + df['左轉-大3'] + df['直行-大4']
    df['天府路_往西_大型車'] = df['左轉-大1'] + df['直行-大2'] + df['右轉-大3']    
    
    df['海濱路_往北_特種車'] = df['直行-特1'] + df['右轉-特2'] + df['左轉-特4']
    df['海濱路_往南_特種車'] = df['左轉-特2'] + df['直行-特3'] + df['右轉-特4']
    df['天府路_往東_特種車'] = df['右轉-特1'] + df['左轉-特3'] + df['直行-特4']
    df['天府路_往西_特種車'] = df['左轉-特1'] + df['直行-特2'] + df['右轉-特3']
    
    ## 統計各方向車流
        
    df['海濱路_往北_PCU'] = df['海濱路_往北_機踏車'] * coe_of_bike\
                         + df['海濱路_往北_小型車'] * coe_of_vehicle\
                         + df['海濱路_往北_大型車'] * coe_of_truck\
                         + df['海濱路_往北_特種車'] * coe_of_special
    df['海濱路_往南_PCU'] = df['海濱路_往南_機踏車'] * coe_of_bike\
                         + df['海濱路_往南_小型車'] * coe_of_vehicle\
                         + df['海濱路_往南_大型車'] * coe_of_truck\
                         + df['海濱路_往南_特種車'] * coe_of_special
    df['天府路_往東_PCU'] = df['天府路_往東_機踏車'] * coe_of_bike\
                         + df['天府路_往東_小型車'] * coe_of_vehicle\
                         + df['天府路_往東_大型車'] * coe_of_truck\
                         + df['天府路_往東_特種車'] * coe_of_special
    df['天府路_往西_PCU'] = df['天府路_往西_機踏車'] * coe_of_bike\
                         + df['天府路_往西_小型車'] * coe_of_vehicle\
                         + df['天府路_往西_大型車'] * coe_of_truck\
                         + df['天府路_往西_特種車'] * coe_of_special
    ## 計算各路段PCU 

    df['海濱路_往北_機踏車比例'] = df['海濱路_往北_機踏車'] / (df['海濱路_往北_機踏車'] + df['海濱路_往北_小型車']\
                                             + df['海濱路_往北_大型車'] + df['海濱路_往北_特種車'])
    df['海濱路_往南_機踏車比例'] = df['海濱路_往南_機踏車'] / (df['海濱路_往南_機踏車'] + df['海濱路_往南_小型車']\
                                             + df['海濱路_往南_大型車'] + df['海濱路_往南_特種車'])
    df['天府路_往東_機踏車比例'] = df['天府路_往東_機踏車'] / (df['天府路_往東_機踏車'] + df['天府路_往東_小型車']\
                                             + df['天府路_往東_大型車'] + df['天府路_往東_特種車'])
    df['天府路_往西_機踏車比例'] = df['天府路_往西_機踏車'] / (df['天府路_往西_機踏車'] + df['天府路_往西_小型車']\
                                             + df['天府路_往西_大型車'] + df['天府路_往西_特種車'])
   
    df['海濱路_往北_大型車比例'] = df['海濱路_往北_大型車'] / (df['海濱路_往北_機踏車'] + df['海濱路_往北_小型車']\
                                             + df['海濱路_往北_大型車'] + df['海濱路_往北_特種車'])
    df['海濱路_往南_大型車比例'] = df['海濱路_往南_大型車'] / (df['海濱路_往南_機踏車'] + df['海濱路_往南_小型車']\
                                             + df['海濱路_往南_大型車'] + df['海濱路_往南_特種車'])
    df['天府路_往東_大型車比例'] = df['天府路_往東_大型車'] / (df['天府路_往東_機踏車'] + df['天府路_往東_小型車']\
                                             + df['天府路_往東_大型車'] + df['天府路_往東_特種車'])
    df['天府路_往西_大型車比例'] = df['天府路_往西_大型車'] / (df['天府路_往西_機踏車'] + df['天府路_往西_小型車']\
                                             + df['天府路_往西_大型車'] + df['天府路_往西_特種車'])

    ## 統計機踏車與大型車比例
    
    天府路綠燈秒數 = 27
    天府路_g值 = 27 + 4
    海濱路綠燈秒數 = 13
    海濱路_g值 = 13 + 4
    黃燈秒數 = 3
    全紅燈秒數 = 2
    c1 = 天府路綠燈秒數 + 海濱路綠燈秒數 + 黃燈秒數*2 + 全紅燈秒數*2
    車道寬度1 = (6.9 + 3.4) / 2
    車道寬度2 = ((8.9 / 2) + (8.6 / 2)) / 2
    fm_無 = 0.97
    fm_有 = 1.00
    車道數1 = 2
    車道數2 = 1
    
    df['c1'] = c1
    df['海濱路_Ngyi'] = Ngyi_cal(海濱路_g值, 車道寬度1)
    df['天府路_Ngyi'] = Ngyi_cal(天府路_g值, 車道寬度2)
    
    ## 計算Ngyi，只與g跟車道寬度有關，與方向無關
    
    df['海濱路_Esb'] = Esb_cal(海濱路_g值)
    df['天府路_Esb'] = Esb_cal(天府路_g值)
    
    ## 計算Esb(大車之小車當量)，只與g有關，與方向無關
    
    df['海濱路_往北_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['海濱路_往北_機踏車比例']]
    df['海濱路_往南_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['海濱路_往南_機踏車比例']]
    df['天府路_往東_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['天府路_往東_機踏車比例']]
    df['天府路_往西_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['天府路_往西_機踏車比例']]
    
    ## 計算Esm(機踏車之小車當量)，與各方向機踏車比例有關
    
    df['海濱路_往北_fv'] = 1 / (1 + (df['海濱路_往北_機踏車比例']*(df['海濱路_往北_Esm']-1))\
                           + df['海濱路_往北_大型車比例']*(df['海濱路_Esb']-1))
    df['海濱路_往南_fv'] = 1 / (1 + (df['海濱路_往南_機踏車比例']*(df['海濱路_往南_Esm']-1))\
                           + df['海濱路_往南_大型車比例']*(df['海濱路_Esb']-1))
    df['天府路_往東_fv'] = 1 / (1 + (df['天府路_往東_機踏車比例']*(df['天府路_往東_Esm']-1))\
                           + df['天府路_往東_大型車比例']*(df['天府路_Esb']-1))
    df['天府路_往西_fv'] = 1 / (1 + (df['天府路_往西_機踏車比例']*(df['天府路_往西_Esm']-1))\
                           + df['天府路_往西_大型車比例']*(df['天府路_Esb']-1))
    
    ## 計算fv(車種組成調整因素)，與Esb、Esm及大車、機車比例有關
    
    df['海濱路_往北_c'] = (3600 / df['c1']) * df['海濱路_Ngyi'] * df['海濱路_往北_fv'] * fm_無 * 車道數1
    df['海濱路_往南_c'] = (3600 / df['c1']) * df['海濱路_Ngyi'] * df['海濱路_往南_fv'] * fm_無 * 車道數2
    df['天府路_往東_c'] = (3600 / df['c1']) * df['天府路_Ngyi'] * df['天府路_往東_fv'] * fm_有 * 車道數1
    df['天府路_往西_c'] = (3600 / df['c1']) * df['天府路_Ngyi'] * df['天府路_往西_fv'] * fm_有 * 車道數1
    
    ## 計算各方向c值
    
    df['海濱路_往北_v/c'] = df['海濱路_往北_PCU'] / df['海濱路_往北_c'].mean()
    df['海濱路_往南_v/c'] = df['海濱路_往南_PCU'] / df['海濱路_往南_c'].mean()
    df['天府路_往東_v/c'] = df['天府路_往東_PCU'] / df['天府路_往東_c'].mean()
    df['天府路_往西_v/c'] = df['天府路_往西_PCU'] / df['天府路_往西_c'].mean()
    
    ## 由各方向車流 / 平均c值

## 最後須用excel計算部分數值確認(點有難)
    

In [14]:
def depart_direction_2ndpoint(df):
    df['台15北段_往南_機踏車'] = df['直行-機2']
    df['台15北段_往北_機踏車'] = df['直行-機1'] + df['右轉-機3']
    df['台15南段_往南_機踏車'] = df['直行-機2'] + df['左轉-機3']
    df['台15南段_往北_機踏車'] = df['直行-機1']
    df['延平路_往南_機踏車'] = df['右轉-機3'] + df['左轉-機3']
    df['延平路_往北_機踏車'] = df['右轉-機1'] + df['左轉-機2']
    
    df['台15北段_往南_小型車'] = df['直行-小2']
    df['台15北段_往北_小型車'] = df['直行-小1'] + df['右轉-小3']
    df['台15南段_往南_小型車'] = df['直行-小2'] + df['左轉-小3']
    df['台15南段_往北_小型車'] = df['直行-小1']
    df['延平路_往南_小型車'] = df['右轉-小3'] + df['左轉-小3']
    df['延平路_往北_小型車'] = df['右轉-小1'] + df['左轉-小2']  
    
    df['台15北段_往南_大型車'] = df['直行-大2']
    df['台15北段_往北_大型車'] = df['直行-大1'] + df['右轉-大3']
    df['台15南段_往南_大型車'] = df['直行-大2'] + df['左轉-大3']
    df['台15南段_往北_大型車'] = df['直行-大1']
    df['延平路_往南_大型車'] = df['右轉-大3'] + df['左轉-大3']
    df['延平路_往北_大型車'] = df['右轉-大1'] + df['左轉-大2']   
    
    df['台15北段_往南_特種車'] = df['直行-特2']
    df['台15北段_往北_特種車'] = df['直行-特1'] + df['右轉-特3']
    df['台15南段_往南_特種車'] = df['直行-特2'] + df['左轉-特3']
    df['台15南段_往北_特種車'] = df['直行-特1']
    df['延平路_往南_特種車'] = df['右轉-特3'] + df['左轉-特3']
    df['延平路_往北_特種車'] = df['右轉-特1'] + df['左轉-特2']
    
    df['台15北段_往南_PCU'] = df['台15北段_往南_機踏車'] * coe_of_bike\
                           + df['台15北段_往南_小型車'] * coe_of_vehicle\
                           + df['台15北段_往南_大型車'] * coe_of_truck\
                           + df['台15北段_往南_特種車'] * coe_of_special
    df['台15北段_往北_PCU'] = df['台15北段_往北_機踏車'] * coe_of_bike\
                           + df['台15北段_往北_小型車'] * coe_of_vehicle\
                           + df['台15北段_往北_大型車'] * coe_of_truck\
                           + df['台15北段_往北_特種車'] * coe_of_special
    df['台15南段_往南_PCU'] = df['台15南段_往南_機踏車'] * coe_of_bike\
                           + df['台15南段_往南_小型車'] * coe_of_vehicle\
                           + df['台15南段_往南_大型車'] * coe_of_truck\
                           + df['台15南段_往南_特種車'] * coe_of_special
    df['台15南段_往北_PCU'] = df['台15南段_往北_機踏車'] * coe_of_bike\
                           + df['台15南段_往北_小型車'] * coe_of_vehicle\
                           + df['台15南段_往北_大型車'] * coe_of_truck\
                           + df['台15南段_往北_特種車'] * coe_of_special
    df['延平路_往南_PCU'] = df['延平路_往南_機踏車'] * coe_of_bike\
                         + df['延平路_往南_小型車'] * coe_of_vehicle\
                         + df['延平路_往南_大型車'] * coe_of_truck\
                         + df['延平路_往南_特種車'] * coe_of_special
    df['延平路_往北_PCU'] = df['延平路_往北_機踏車'] * coe_of_bike\
                         + df['延平路_往北_小型車'] * coe_of_vehicle\
                         + df['延平路_往北_大型車'] * coe_of_truck\
                         + df['延平路_往北_特種車'] * coe_of_special
## 台15線及延平路路口

    df['台15北段_往南_機踏車比例'] = df['台15北段_往南_機踏車'] / (df['台15北段_往南_機踏車'] + df['台15北段_往南_小型車']\
                                                 + df['台15北段_往南_大型車'] + df['台15北段_往南_特種車'])
    df['台15北段_往北_機踏車比例'] = df['台15北段_往北_機踏車'] / (df['台15北段_往北_機踏車'] + df['台15北段_往北_小型車']\
                                                 + df['台15北段_往北_大型車'] + df['台15北段_往北_特種車'])
    df['台15南段_往南_機踏車比例'] = df['台15南段_往南_機踏車'] / (df['台15南段_往南_機踏車'] + df['台15南段_往南_小型車']\
                                                 + df['台15南段_往南_大型車'] + df['台15南段_往南_特種車'])
    df['台15南段_往北_機踏車比例'] = df['台15南段_往北_機踏車'] / (df['台15南段_往北_機踏車'] + df['台15南段_往北_小型車']\
                                                 + df['台15南段_往北_大型車'] + df['台15南段_往北_特種車'])
    df['延平路_往南_機踏車比例'] = df['延平路_往南_機踏車'] / (df['延平路_往南_機踏車'] + df['延平路_往南_小型車']\
                                             + df['延平路_往南_大型車'] + df['延平路_往南_特種車'])
    df['延平路_往北_機踏車比例'] = df['延平路_往北_機踏車'] / (df['延平路_往北_機踏車'] + df['延平路_往北_小型車']\
                                             + df['延平路_往北_大型車'] + df['延平路_往北_特種車'])
   
    df['台15北段_往南_大型車比例'] = df['台15北段_往南_大型車'] / (df['台15北段_往南_機踏車'] + df['台15北段_往南_小型車']\
                                                 + df['台15北段_往南_大型車'] + df['台15北段_往南_特種車'])
    df['台15北段_往北_大型車比例'] = df['台15北段_往北_大型車'] / (df['台15北段_往北_機踏車'] + df['台15北段_往北_小型車']\
                                                 + df['台15北段_往北_大型車'] + df['台15北段_往北_特種車'])
    df['台15南段_往南_大型車比例'] = df['台15南段_往南_大型車'] / (df['台15南段_往南_機踏車'] + df['台15南段_往南_小型車']\
                                                 + df['台15南段_往南_大型車'] + df['台15南段_往南_特種車'])
    df['台15南段_往北_大型車比例'] = df['台15南段_往北_大型車'] / (df['台15南段_往北_機踏車'] + df['台15南段_往北_小型車']\
                                                 + df['台15南段_往北_大型車'] + df['台15南段_往北_特種車'])
    df['延平路_往南_大型車比例'] = df['延平路_往南_大型車'] / (df['延平路_往南_機踏車'] + df['延平路_往南_小型車']\
                                             + df['延平路_往南_大型車'] + df['延平路_往南_特種車'])
    df['延平路_往北_大型車比例'] = df['延平路_往北_大型車'] / (df['延平路_往北_機踏車'] + df['延平路_往北_小型車']\
                                             + df['延平路_往北_大型車'] + df['延平路_往北_特種車'])
    ## 統計機踏車與大型車比例

    台15北段綠燈秒數 = 72
    台15北段_g值 = 台15北段綠燈秒數 + 4
    台15南段綠燈秒數 = 72
    台15南段_g值 = 台15南段綠燈秒數 + 4
    延平路綠燈秒數 = 45
    延平路_g值 = 延平路綠燈秒數 + 4
    黃燈秒數 = 3
    全紅燈秒數 = 2
    c2 = 台15北段綠燈秒數 + 延平路綠燈秒數 + 黃燈秒數*2 + 全紅燈秒數*2
    車道寬度1 = 10.5 / 2
    車道寬度2 = 8.3 / 2
    車道寬度3 = 3.8
    fm_無 = 0.97
    fm_有 = 1.00
    車道數1 = 2
    車道數2 = 1
    
    ## 車道寬度參考SGS數據路寬計算
    
    df['c1'] = c2
    df['台15北段_Ngyi'] = Ngyi_cal(台15北段_g值, 車道寬度1)
    df['台15南段_Ngyi'] = Ngyi_cal(台15南段_g值, 車道寬度2)
    df['延平路_Ngyi'] = Ngyi_cal(延平路_g值, 車道寬度3)
    
    ## 計算Ngyi，只與g跟車道寬度有關，與方向無關
    
    df['台15北段_Esb'] = Esb_cal(台15北段_g值)
    df['台15南段_Esb'] = Esb_cal(台15南段_g值)
    df['延平路_Esb'] = Esb_cal(延平路_g值)
    
    ## 計算Esb(大車之小車當量)，只與g有關，與方向無關
    
    df['台15北段_往南_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台15北段_往南_機踏車比例']]
    df['台15北段_往北_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台15北段_往北_機踏車比例']]
    df['台15南段_往南_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台15南段_往南_機踏車比例']]
    df['台15南段_往北_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台15南段_往北_機踏車比例']]
    df['延平路_往南_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['延平路_往南_機踏車比例']]
    df['延平路_往北_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['延平路_往北_機踏車比例']]    
    
    ## 計算Esm(機踏車之小車當量)，與各方向機踏車比例有關
    
    df['台15北段_往南_fv'] = 1 / (1 + (df['台15北段_往南_機踏車比例']*(df['台15北段_往南_Esm']-1))\
                             + df['台15北段_往南_大型車比例']*(df['台15北段_Esb']-1))
    df['台15北段_往北_fv'] = 1 / (1 + (df['台15北段_往北_機踏車比例']*(df['台15北段_往北_Esm']-1))\
                             + df['台15北段_往北_大型車比例']*(df['台15北段_Esb']-1))
    df['台15南段_往南_fv'] = 1 / (1 + (df['台15南段_往南_機踏車比例']*(df['台15南段_往南_Esm']-1))\
                             + df['台15南段_往南_大型車比例']*(df['台15南段_Esb']-1))
    df['台15南段_往北_fv'] = 1 / (1 + (df['台15南段_往北_機踏車比例']*(df['台15南段_往北_Esm']-1))\
                             + df['台15南段_往北_大型車比例']*(df['台15南段_Esb']-1))
    df['延平路_往南_fv'] = 1 / (1 + (df['延平路_往南_機踏車比例']*(df['延平路_往南_Esm']-1))\
                           + df['延平路_往南_大型車比例']*(df['延平路_Esb']-1))
    df['延平路_往北_fv'] = 1 / (1 + (df['延平路_往北_機踏車比例']*(df['延平路_往北_Esm']-1))\
                           + df['延平路_往北_大型車比例']*(df['延平路_Esb']-1))
    
    ## 計算fv(車種組成調整因素)，與Esb、Esm及大車、機車比例有關
    
    df['台15北段_往南_c'] = (3600 / df['c1']) * df['台15北段_Ngyi'] * df['台15北段_往南_fv'] * fm_有 * 車道數1
    df['台15北段_往北_c'] = (3600 / df['c1']) * df['台15北段_Ngyi'] * df['台15北段_往北_fv'] * fm_有 * 車道數1
    df['台15南段_往南_c'] = (3600 / df['c1']) * df['台15南段_Ngyi'] * df['台15南段_往南_fv'] * fm_有 * 車道數1
    df['台15南段_往北_c'] = (3600 / df['c1']) * df['台15南段_Ngyi'] * df['台15南段_往北_fv'] * fm_有 * 車道數1
    df['延平路_往南_c'] = (3600 / df['c1']) * df['延平路_Ngyi'] * df['延平路_往南_fv'] * fm_無 * 車道數2
    df['延平路_往北_c'] = (3600 / df['c1']) * df['延平路_Ngyi'] * df['延平路_往北_fv'] * fm_無 * 車道數2
    
    
    ## 計算各方向c值
    
    df['台15北段_往南_v/c'] = df['台15北段_往南_PCU'] / df['台15北段_往南_c'].mean()
    df['台15北段_往北_v/c'] = df['台15北段_往北_PCU'] / df['台15北段_往北_c'].mean()
    df['台15南段_往南_v/c'] = df['台15南段_往南_PCU'] / df['台15南段_往南_c'].mean()
    df['台15南段_往北_v/c'] = df['台15南段_往北_PCU'] / df['台15南段_往北_c'].mean()
    df['延平路_往南_v/c'] = df['延平路_往南_PCU'] / df['延平路_往南_c'].mean()
    df['延平路_往北_v/c'] = df['延平路_往北_PCU'] / df['延平路_往北_c'].mean()
    
    ## 由各方向車流 / 平均c值

## 最後須用excel計算部分數值確認(點有難)
    

In [15]:
def depart_direction_3thpoint(df):
    df['台68_往東_機踏車'] = df['直行-機1'] + df['右轉-機3']
    df['台68_往西_機踏車'] = df['直行-機2']
    df['榮濱路_往南_機踏車'] = df['右轉-機1'] + df['左轉-機2']
    df['榮濱路_往北_機踏車'] = df['右轉-機3'] + df['左轉-機3']
    df['南寮大道_往東_機踏車'] = df['直行-機1']
    df['南寮大道_往西_機踏車'] = df['直行-機2'] + df['左轉-機3']

    df['台68_往東_小型車'] = df['直行-小1'] + df['右轉-小3']
    df['台68_往西_小型車'] = df['直行-小2']
    df['榮濱路_往南_小型車'] = df['右轉-小1'] + df['左轉-小2']
    df['榮濱路_往北_小型車'] = df['右轉-小3'] + df['左轉-小3']
    df['南寮大道_往東_小型車'] = df['直行-小1']
    df['南寮大道_往西_小型車'] = df['直行-小2'] + df['左轉-小3']
    
    df['台68_往東_大型車'] = df['直行-大1'] + df['右轉-大3']
    df['台68_往西_大型車'] = df['直行-大2']
    df['榮濱路_往南_大型車'] = df['右轉-大1'] + df['左轉-大2']
    df['榮濱路_往北_大型車'] = df['右轉-大3'] + df['左轉-大3']
    df['南寮大道_往東_大型車'] = df['直行-大1']
    df['南寮大道_往西_大型車'] = df['直行-大2'] + df['左轉-大3']
    
    df['台68_往東_特種車'] = df['直行-特1'] + df['右轉-特3']
    df['台68_往西_特種車'] = df['直行-特2']
    df['榮濱路_往南_特種車'] = df['右轉-特1'] + df['左轉-特2']
    df['榮濱路_往北_特種車'] = df['右轉-特3'] + df['左轉-特3']
    df['南寮大道_往東_特種車'] = df['直行-特1']
    df['南寮大道_往西_特種車'] = df['直行-特2'] + df['左轉-特3']
    
    df['台68_往東_PCU'] = df['台68_往東_機踏車'] * coe_of_bike\
                       + df['台68_往東_小型車'] * coe_of_vehicle\
                       + df['台68_往東_大型車'] * coe_of_truck\
                       + df['台68_往東_特種車'] * coe_of_special
    df['台68_往西_PCU'] = df['台68_往西_機踏車'] * coe_of_bike\
                       + df['台68_往西_小型車'] * coe_of_vehicle\
                       + df['台68_往西_大型車'] * coe_of_truck\
                       + df['台68_往西_特種車'] * coe_of_special
    df['榮濱路_往南_PCU'] = df['榮濱路_往南_機踏車'] * coe_of_bike\
                         + df['榮濱路_往南_小型車'] * coe_of_vehicle\
                         + df['榮濱路_往南_大型車'] * coe_of_truck\
                         + df['榮濱路_往南_特種車'] * coe_of_special
    df['榮濱路_往北_PCU'] = df['榮濱路_往北_機踏車'] * coe_of_bike\
                         + df['榮濱路_往北_小型車'] * coe_of_vehicle\
                         + df['榮濱路_往北_大型車'] * coe_of_truck\
                         + df['榮濱路_往北_特種車'] * coe_of_special
    df['南寮大道_往東_PCU'] = df['南寮大道_往東_機踏車'] * coe_of_bike\
                           + df['南寮大道_往東_小型車'] * coe_of_vehicle\
                           + df['南寮大道_往東_大型車'] * coe_of_truck\
                           + df['南寮大道_往東_特種車'] * coe_of_special
    df['南寮大道_往西_PCU'] = df['南寮大道_往西_機踏車'] * coe_of_bike\
                           + df['南寮大道_往西_小型車'] * coe_of_vehicle\
                           + df['南寮大道_往西_大型車'] * coe_of_truck\
                           + df['南寮大道_往西_特種車'] * coe_of_special
    ## 南寮大道及榮濱路路口

    df['台68_往東_機踏車比例'] = df['台68_往東_機踏車'] / (df['台68_往東_機踏車'] + df['台68_往東_小型車'] + df['台68_往東_大型車'] + df['台68_往東_特種車'])
    df['台68_往西_機踏車比例'] = df['台68_往西_機踏車'] / (df['台68_往西_機踏車'] + df['台68_往西_小型車'] + df['台68_往西_大型車'] + df['台68_往西_特種車'])
    df['榮濱路_往南_機踏車比例'] = df['榮濱路_往南_機踏車'] / (df['榮濱路_往南_機踏車'] + df['榮濱路_往南_小型車'] + df['榮濱路_往南_大型車'] + df['榮濱路_往南_特種車'])
    df['榮濱路_往北_機踏車比例'] = df['榮濱路_往北_機踏車'] / (df['榮濱路_往北_機踏車'] + df['榮濱路_往北_小型車'] + df['榮濱路_往北_大型車'] + df['榮濱路_往北_特種車'])
    df['南寮大道_往東_機踏車比例'] = df['南寮大道_往東_機踏車'] / (df['南寮大道_往東_機踏車'] + df['南寮大道_往東_小型車'] + df['南寮大道_往東_大型車'] + df['南寮大道_往東_特種車'])
    df['南寮大道_往西_機踏車比例'] = df['南寮大道_往西_機踏車'] / (df['南寮大道_往西_機踏車'] + df['南寮大道_往西_小型車'] + df['南寮大道_往西_大型車'] + df['南寮大道_往西_特種車'])
   
    df['台68_往東_大型車比例'] = df['台68_往東_大型車'] / (df['台68_往東_機踏車'] + df['台68_往東_小型車'] + df['台68_往東_大型車'] + df['台68_往東_特種車'])
    df['台68_往西_大型車比例'] = df['台68_往西_大型車'] / (df['台68_往西_機踏車'] + df['台68_往西_小型車'] + df['台68_往西_大型車'] + df['台68_往西_特種車'])
    df['榮濱路_往南_大型車比例'] = df['榮濱路_往南_大型車'] / (df['榮濱路_往南_機踏車'] + df['榮濱路_往南_小型車'] + df['榮濱路_往南_大型車'] + df['榮濱路_往南_特種車'])
    df['榮濱路_往北_大型車比例'] = df['榮濱路_往北_大型車'] / (df['榮濱路_往北_機踏車'] + df['榮濱路_往北_小型車'] + df['榮濱路_往北_大型車'] + df['榮濱路_往北_特種車'])
    df['南寮大道_往東_大型車比例'] = df['南寮大道_往東_大型車'] / (df['南寮大道_往東_機踏車'] + df['南寮大道_往東_小型車'] + df['南寮大道_往東_大型車'] + df['南寮大道_往東_特種車'])
    df['南寮大道_往西_大型車比例'] = df['南寮大道_往西_大型車'] / (df['南寮大道_往西_機踏車'] + df['南寮大道_往西_小型車'] + df['南寮大道_往西_大型車'] + df['南寮大道_往西_特種車'])

    ## 統計機踏車與大型車比例


    台68綠燈秒數 = 35
    台68_g值 = 台68綠燈秒數 + 4
    榮濱路綠燈秒數 = 22
    榮濱路_g值 = 榮濱路綠燈秒數 + 4
    南寮大道綠燈秒數 = 35
    南寮大道_g值 = 南寮大道綠燈秒數 + 4
    黃燈秒數 = 3
    全紅燈秒數 = 2
    c3 = 榮濱路綠燈秒數 + 南寮大道綠燈秒數 + 黃燈秒數*2 + 全紅燈秒數*2
    車道寬度1 = 7.4 / 2
    車道寬度2 = 6.1 / 2
    車道寬度3 = 7.2 / 2
    fm_無 = 0.97
    fm_有 = 1.00
    車道數 = 2
    
    df['c1'] = c3
    df['台68_Ngyi'] = Ngyi_cal(台68_g值, 車道寬度1)
    df['榮濱路_Ngyi'] = Ngyi_cal(榮濱路_g值, 車道寬度2)
    df['南寮大道_Ngyi'] = Ngyi_cal(南寮大道_g值, 車道寬度3)
    
    ## 計算Ngyi，只與g跟車道寬度有關，與方向無關
    
    df['台68_Esb'] = Esb_cal(台68_g值)
    df['榮濱路_Esb'] = Esb_cal(榮濱路_g值)
    df['南寮大道_Esb'] = Esb_cal(南寮大道_g值)
    
    ## 計算Esb(大車之小車當量)，只與g有關，與方向無關
    
    df['台68_往東_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台68_往東_機踏車比例']]
    df['台68_往西_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['台68_往西_機踏車比例']]
    df['榮濱路_往南_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['榮濱路_往南_機踏車比例']]
    df['榮濱路_往北_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['榮濱路_往北_機踏車比例']]
    df['南寮大道_往東_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['南寮大道_往東_機踏車比例']]
    df['南寮大道_往西_Esm'] = [0.23 + 0.1556*np.exp(-x*100/10.6396) + 0.3673*np.exp(-x*100/67.0339) for x in df['南寮大道_往西_機踏車比例']]    
    
    ## 計算Esm(機踏車之小車當量)，與各方向機踏車比例有關
    
    df['台68_往東_fv'] = 1 / (1 + (df['台68_往東_機踏車比例']*(df['台68_往東_Esm']-1)) + df['台68_往東_大型車比例']*(df['台68_Esb']-1))
    df['台68_往西_fv'] = 1 / (1 + (df['台68_往西_機踏車比例']*(df['台68_往西_Esm']-1)) + df['台68_往西_大型車比例']*(df['台68_Esb']-1))
    df['榮濱路_往南_fv'] = 1 / (1 + (df['榮濱路_往南_機踏車比例']*(df['榮濱路_往南_Esm']-1)) + df['榮濱路_往南_大型車比例']*(df['榮濱路_Esb']-1))
    df['榮濱路_往北_fv'] = 1 / (1 + (df['榮濱路_往北_機踏車比例']*(df['榮濱路_往北_Esm']-1)) + df['榮濱路_往北_大型車比例']*(df['榮濱路_Esb']-1))
    df['南寮大道_往東_fv'] = 1 / (1 + (df['南寮大道_往東_機踏車比例']*(df['南寮大道_往東_Esm']-1)) + df['南寮大道_往東_大型車比例']*(df['南寮大道_Esb']-1))
    df['南寮大道_往西_fv'] = 1 / (1 + (df['南寮大道_往西_機踏車比例']*(df['南寮大道_往西_Esm']-1)) + df['南寮大道_往西_大型車比例']*(df['南寮大道_Esb']-1))
    ## 計算fv(車種組成調整因素)，與Esb、Esm及大車、機車比例有關
    
    df['台68_往東_c'] = (3600 / df['c1']) * df['台68_Ngyi'] * df['台68_往東_fv'] * fm_有 * 車道數
    df['台68_往西_c'] = (3600 / df['c1']) * df['台68_Ngyi'] * df['台68_往西_fv'] * fm_有 * 車道數 
    df['榮濱路_往南_c'] = (3600 / df['c1']) * df['榮濱路_Ngyi'] * df['榮濱路_往南_fv'] * fm_無 * 車道數
    df['榮濱路_往北_c'] = (3600 / df['c1']) * df['榮濱路_Ngyi'] * df['榮濱路_往北_fv'] * fm_無 * 車道數
    df['南寮大道_往東_c'] = (3600 / df['c1']) * df['南寮大道_Ngyi'] * df['南寮大道_往東_fv'] * fm_無 * 車道數
    df['南寮大道_往西_c'] = (3600 / df['c1']) * df['南寮大道_Ngyi'] * df['南寮大道_往西_fv'] * fm_無 * 車道數 
    
    
    ## 計算各方向c值
    
    df['台68_往東_v/c'] = df['台68_往東_PCU'] / df['台68_往東_c'].mean()
    df['台68_往西_v/c'] = df['台68_往西_PCU'] / df['台68_往西_c'].mean()
    df['榮濱路_往南_v/c'] = df['榮濱路_往南_PCU'] / df['榮濱路_往南_c'].mean()
    df['榮濱路_往北_v/c'] = df['榮濱路_往北_PCU'] / df['榮濱路_往北_c'].mean()
    df['南寮大道_往東_v/c'] = df['南寮大道_往東_PCU'] / df['南寮大道_往東_c'].mean()
    df['南寮大道_往西_v/c'] = df['南寮大道_往西_PCU'] / df['南寮大道_往西_c'].mean()
    
    ## 由各方向車流 / 平均c值

## 最後須用excel計算部分數值確認(點有難)



In [16]:
depart_direction_1stpoint(df1_woday)
depart_direction_1stpoint(df1_hoday)
depart_direction_2ndpoint(df2_woday)
depart_direction_2ndpoint(df2_hoday)
depart_direction_3thpoint(df3_woday)
depart_direction_3thpoint(df3_hoday)

In [17]:
df2_woday.replace('*', 0, inplace=True)
df2_hoday.replace('*', 0, inplace=True)
df3_woday.replace('*', 0, inplace=True)
df3_hoday.replace('*', 0, inplace=True)

## 可不需要此動作，但可方便計算總數

In [18]:
#lists_of_sum = [df1_woday.sum().sum() / 2, df1_hoday.sum().sum() / 2, df2_woday.sum().sum() / 2,
                #df2_hoday.sum().sum() / 2, df3_woday.sum().sum() / 2, df3_hoday.sum().sum() / 2]
#print(lists_of_sum)

#測試總車輛數是否正確(8/12 dataframe新增計算PCU，故總量會增加PCU的數量)
#需除以2，因原本rawdata還在

## 0812 ==> 各路段統計方式更新，有些方向會有重複計算的地方，所以總量不會一樣

In [19]:
#df1_woday.head()
#測試OK

In [20]:
#df1_hoday.head()
#測試OK

In [21]:
lists_of_index = ['00-01', '01-02', '02-03', '03-04', '04-05', '05-06', '06-07', '07-08',
                  '08-09', '09-10', '10-11', '11-12', '12-13', '13-14', '14-15', '15-16',
                  '16-17', '17-18', '18-19', '19-20', '20-21', '21-22', '22-23', '23-24']

In [22]:
df1_woday.index = lists_of_index
df1_hoday.index = lists_of_index
df2_woday.index = lists_of_index
df2_hoday.index = lists_of_index
df3_woday.index = lists_of_index
df3_hoday.index = lists_of_index

##替換index

In [23]:
#writer = pd.ExcelWriter('新竹海淡_交通量整理(方向).xlsx')
#df1_woday.to_excel(writer, sheet_name='天府路及海濱路路口(平日)')
#df1_hoday.to_excel(writer, sheet_name='天府路及海濱路路口(假日)')
#df2_woday.to_excel(writer, sheet_name='台15線及延平路口(平日)')
#df2_hoday.to_excel(writer, sheet_name='台15線及延平路口(假日)')
#df3_woday.to_excel(writer,sheet_name='南寮大道及榮濱路(平日)')
#df3_hoday.to_excel(writer,sheet_name='南寮大道及榮濱路(假日)')

#writer.save()
#writer.close()
#df2_woday.to

## 若要寫在同一檔案中不同sheet，需寫在同一欄裡，寫在不同欄，會已最後的code會覆寫

In [24]:
PCU_1 = ['海濱路_往北_PCU', '海濱路_往南_PCU', '天府路_往東_PCU', '天府路_往西_PCU']
PCU_2 = ['台15北段_往南_PCU', '台15北段_往北_PCU', '台15南段_往南_PCU', '台15南段_往北_PCU', 
               '延平路_往南_PCU', '延平路_往北_PCU']
PCU_3 = ['台68_往東_PCU', '台68_往西_PCU', '榮濱路_往南_PCU', '榮濱路_往北_PCU',
               '南寮大道_往東_PCU', '南寮大道_往西_PCU']

c_1 = ['海濱路_往北_c', '海濱路_往南_c', '天府路_往東_c', '天府路_往西_c']
c_2 = ['台15北段_往南_c', '台15北段_往北_c', '台15南段_往南_c', '台15南段_往北_c', 
               '延平路_往南_c', '延平路_往北_c']
c_3 = ['台68_往東_c', '台68_往西_c', '榮濱路_往南_c', '榮濱路_往北_c',
               '南寮大道_往東_c', '南寮大道_往西_c']

vc_1 = ['海濱路_往北_v/c', '海濱路_往南_v/c', '天府路_往東_v/c', '天府路_往西_v/c']
vc_2 = ['台15北段_往南_v/c', '台15北段_往北_v/c', '台15南段_往南_v/c', '台15南段_往北_v/c', 
               '延平路_往南_v/c', '延平路_往北_v/c']
vc_3 = ['台68_往東_v/c', '台68_往西_v/c', '榮濱路_往南_v/c', '榮濱路_往北_v/c',
               '南寮大道_往東_v/c', '南寮大道_往西_v/c']

In [25]:
df_allwo = [df1_woday, df1_woday, df1_woday, df1_woday,
           df2_woday, df2_woday, df2_woday, df2_woday, df2_woday, df2_woday,
           df3_woday, df3_woday, df3_woday, df3_woday, df3_woday, df3_woday]

df_allho = [df1_hoday, df1_hoday, df1_hoday, df1_hoday,
           df2_hoday, df2_hoday, df2_hoday, df2_hoday, df2_hoday, df2_hoday,
           df3_hoday, df3_hoday, df3_hoday, df3_hoday, df3_hoday, df3_hoday]

len(df_allwo)

16

In [26]:
PCU_all = PCU_1 + PCU_2 + PCU_3
c_all = c_1 + c_2 + c_3
vc_all = vc_1 + vc_2 +vc_3
len(PCU_all)

16

In [27]:
if df1_woday.loc['00-01':'11-12', '海濱路_往北_v/c'].max() <= 0.37:
    print('A')
elif df1_woday.loc['00-01':'11-12', '海濱路_往北_v/c'].max() <= 0.62:
    print('B')
elif df1_woday.loc['00-01':'11-12', '海濱路_往北_v/c'].max() <= 0.79:
    print('C')
elif df1_woday.loc['00-01':'11-12', '海濱路_往北_v/c'].max() <= 0.91:
    print('D')
elif df1_woday.loc['00-01':'11-12', '海濱路_往北_v/c'].max() <= 1.00:
    print('E')
else:
    print('F')

A


In [28]:
def 服務水準判斷_晨峰(df, col):
    if df.loc['00-01':'11-12', col].max() <= 0.37:
        return 'A'
    elif df.loc['00-01':'11-12', col].max() <= 0.62:
        return 'B'
    elif df.loc['00-01':'11-12', col].max() <= 0.79:
        return 'C'
    elif df.loc['00-01':'11-12', col].max() <= 0.91:
        return 'D'
    elif df.loc['00-01':'11-12', col].max() <= 1.00:
        return 'E'
    else:
        return 'F'

In [29]:
def 服務水準判斷_昏峰(df, col):
    if df.loc['12-13':'23-24', col].max() <= 0.37:
        return 'A'
    elif df.loc['12-13':'23-24', col].max() <= 0.62:
        return 'B'
    elif df.loc['12-13':'23-24', col].max() <= 0.79:
        return 'C'
    elif df.loc['12-13':'23-24', col].max() <= 0.91:
        return 'D'
    elif df.loc['12-13':'23-24', col].max() <= 1.00:
        return 'E'
    else:
        return 'F'

In [30]:
for df, col_name in zip(df_allwo, vc_all):
    服務水準判斷_晨峰(df, col_name)
    print(服務水準判斷_晨峰(df, col_name))
    
## 平日晨峰服務水準

A
A
A
A
B
C
B
C
B
B
A
A
A
A
A
A


In [31]:
for df, col_name in zip(df_allwo, vc_all):
    服務水準判斷_昏峰(df, col_name)
    print(服務水準判斷_昏峰(df, col_name))

## 平日昏峰服務水準

A
A
A
A
B
B
B
B
B
B
A
A
A
A
A
A


In [32]:
for df, col_name in zip(df_allho, vc_all):
    服務水準判斷_晨峰(df, col_name)
    print(服務水準判斷_晨峰(df, col_name))
    
## 假日晨峰服務水準

A
A
A
A
A
A
B
A
A
A
A
A
A
A
A
A


In [33]:
for df, col_name in zip(df_allho, vc_all):
    服務水準判斷_昏峰(df, col_name)
    print(服務水準判斷_昏峰(df, col_name))
    
## 平日昏峰服務水準   

A
A
A
A
A
B
B
B
A
A
B
A
B
A
A
A


In [34]:
rushhour_day_PCU_wo = {}
rushhour_night_PCU_wo = {}
for df, col_name in zip(df_allwo, PCU_all):
    temp = [df.loc['00-01':'11-12', col_name].idxmax(), df.loc['00-01':'11-12', col_name].max()]
    rushhour_day_PCU_wo[col_name] = temp
for df, col_name in zip(df_allwo, PCU_all):
    temp = [df.loc['12-13':'23-24', col_name].idxmax(), df.loc['12-13':'23-24', col_name].max()]
    rushhour_night_PCU_wo[col_name] = temp

    ## 最大PCU
    
rushhour_c_wo = {}
for df, col_name in zip(df_allwo, c_all):
    temp = [df.loc['00-01':'23-24', col_name].mean()]
    rushhour_c_wo[col_name] = temp
    
    ## C平均，無晨昏峰之分

rushhour_day_vc_wo = {}
rushhour_night_vc_wo = {}
for df, col_name in zip(df_allwo, vc_all):
    temp = [df.loc['00-01':'11-12', col_name].idxmax(), df.loc['00-01':'11-12', col_name].max()]
    rushhour_day_vc_wo[col_name] = temp
for df, col_name in zip(df_allwo, vc_all):
    temp = [df.loc['12-13':'23-24', col_name].idxmax(), df.loc['12-13':'23-24', col_name].max()]
    rushhour_night_vc_wo[col_name] = temp
    
    ## 最大v/c

## 平日

In [35]:
rushhour_day_PCU_ho = {}
rushhour_night_PCU_ho = {}
for df, col_name in zip(df_allho, PCU_all):
    temp = [df.loc['00-01':'11-12', col_name].idxmax(), df.loc['00-01':'11-12', col_name].max()]
    rushhour_day_PCU_ho[col_name] = temp
for df, col_name in zip(df_allho, PCU_all):
    temp = [df.loc['12-13':'23-24', col_name].idxmax(), df.loc['12-13':'23-24', col_name].max()]
    rushhour_night_PCU_ho[col_name] = temp

    ## 最大PCU
    
rushhour_c_ho = {}
for df, col_name in zip(df_allho, c_all):
    temp = [df.loc['00-01':'23-24', col_name].mean()]
    rushhour_c_ho[col_name] = temp
    
    ## C平均，無晨昏峰之分

rushhour_day_vc_ho = {}
rushhour_night_vc_ho = {}
for df, col_name in zip(df_allho, vc_all):
    temp = [df.loc['00-01':'11-12', col_name].idxmax(), df.loc['00-01':'11-12', col_name].max()]
    rushhour_day_vc_ho[col_name] = temp
for df, col_name in zip(df_allho, vc_all):
    temp = [df.loc['12-13':'23-24', col_name].idxmax(), df.loc['12-13':'23-24', col_name].max()]
    rushhour_night_vc_ho[col_name] = temp
    
    ##最大v/c
    
## 假日

In [36]:
rushhour_day_PCU_wo,rushhour_day_PCU_ho

({'海濱路_往北_PCU': ['08-09', 182.5],
  '海濱路_往南_PCU': ['08-09', 149.0],
  '天府路_往東_PCU': ['10-11', 77.0],
  '天府路_往西_PCU': ['09-10', 135.0],
  '台15北段_往南_PCU': ['11-12', 1294.5],
  '台15北段_往北_PCU': ['08-09', 1582.0],
  '台15南段_往南_PCU': ['11-12', 1487.5],
  '台15南段_往北_PCU': ['08-09', 1579.0],
  '延平路_往南_PCU': ['08-09', 426.0],
  '延平路_往北_PCU': ['08-09', 498.5],
  '台68_往東_PCU': ['08-09', 379.5],
  '台68_往西_PCU': ['09-10', 76.5],
  '榮濱路_往南_PCU': ['09-10', 280.0],
  '榮濱路_往北_PCU': ['08-09', 341.0],
  '南寮大道_往東_PCU': ['11-12', 55.0],
  '南寮大道_往西_PCU': ['09-10', 82.5]},
 {'海濱路_往北_PCU': ['06-07', 121.0],
  '海濱路_往南_PCU': ['10-11', 110.0],
  '天府路_往東_PCU': ['09-10', 97.5],
  '天府路_往西_PCU': ['11-12', 185.0],
  '台15北段_往南_PCU': ['10-11', 748.5],
  '台15北段_往北_PCU': ['11-12', 574.5],
  '台15南段_往南_PCU': ['10-11', 942.0],
  '台15南段_往北_PCU': ['11-12', 572.5],
  '延平路_往南_PCU': ['11-12', 201.5],
  '延平路_往北_PCU': ['09-10', 194.0],
  '台68_往東_PCU': ['11-12', 318.5],
  '台68_往西_PCU': ['11-12', 223.0],
  '榮濱路_往南_PCU': ['11-12', 278.

In [37]:
rushhour_night_PCU_wo, rushhour_night_PCU_ho

({'海濱路_往北_PCU': ['17-18', 214.0],
  '海濱路_往南_PCU': ['17-18', 140.5],
  '天府路_往東_PCU': ['18-19', 88.5],
  '天府路_往西_PCU': ['17-18', 237.5],
  '台15北段_往南_PCU': ['18-19', 1024.5],
  '台15北段_往北_PCU': ['17-18', 1003.0],
  '台15南段_往南_PCU': ['17-18', 1477.0],
  '台15南段_往北_PCU': ['17-18', 999.5],
  '延平路_往南_PCU': ['17-18', 459.5],
  '延平路_往北_PCU': ['17-18', 403.0],
  '台68_往東_PCU': ['17-18', 368.0],
  '台68_往西_PCU': ['16-17', 134.0],
  '榮濱路_往南_PCU': ['17-18', 343.5],
  '榮濱路_往北_PCU': ['17-18', 260.5],
  '南寮大道_往東_PCU': ['17-18', 121.0],
  '南寮大道_往西_PCU': ['16-17', 151.5]},
 {'海濱路_往北_PCU': ['17-18', 269.5],
  '海濱路_往南_PCU': ['17-18', 118.5],
  '天府路_往東_PCU': ['17-18', 294.0],
  '天府路_往西_PCU': ['18-19', 335.5],
  '台15北段_往南_PCU': ['17-18', 817.5],
  '台15北段_往北_PCU': ['17-18', 1315.0],
  '台15南段_往南_PCU': ['17-18', 1036.0],
  '台15南段_往北_PCU': ['17-18', 1309.5],
  '延平路_往南_PCU': ['16-17', 231.0],
  '延平路_往北_PCU': ['17-18', 290.0],
  '台68_往東_PCU': ['19-20', 789.5],
  '台68_往西_PCU': ['16-17', 544.0],
  '榮濱路_往南_PCU': ['16-17'

In [38]:
rushhour_c_wo, rushhour_c_ho

({'海濱路_往北_c': [1319.5539646720674],
  '海濱路_往南_c': [606.9250918148789],
  '天府路_往東_c': [2578.5353315328953],
  '天府路_往西_c': [2507.649853942775],
  '台15北段_往南_c': [2402.8981233916306],
  '台15北段_往北_c': [2366.1267673530538],
  '台15南段_往南_c': [2461.564872973838],
  '台15南段_往北_c': [2313.6283147369345],
  '延平路_往南_c': [819.2285278162781],
  '延平路_往北_c': [833.5792606034069],
  '台68_往東_c': [1940.6448486410534],
  '台68_往西_c': [1960.9902834185486],
  '榮濱路_往南_c': [1182.2529087855642],
  '榮濱路_往北_c': [1190.8363456073587],
  '南寮大道_往東_c': [1894.8523801735082],
  '南寮大道_往西_c': [1975.3965634141878]},
 {'海濱路_往北_c': [1254.1956592817185],
  '海濱路_往南_c': [661.2081334407764],
  '天府路_往東_c': [2508.3746211563034],
  '天府路_往西_c': [2483.55793271036],
  '台15北段_往南_c': [2494.5299100606435],
  '台15北段_往北_c': [2468.150169441981],
  '台15南段_往南_c': [2523.542223028908],
  '台15南段_往北_c': [2407.3694142449776],
  '延平路_往南_c': [833.5144223484758],
  '延平路_往北_c': [814.1126155688734],
  '台68_往東_c': [1939.329204098458],
  '台68_往西_c': [1935.01

In [39]:
rushhour_day_vc_wo, rushhour_day_vc_ho

({'海濱路_往北_v/c': ['08-09', 0.13830430955156464],
  '海濱路_往南_v/c': ['08-09', 0.24549981869170634],
  '天府路_往東_v/c': ['10-11', 0.029861913877373483],
  '天府路_往西_v/c': ['09-10', 0.053835267227495764],
  '台15北段_往南_v/c': ['11-12', 0.5387244625139769],
  '台15北段_往北_v/c': ['08-09', 0.6686032303204772],
  '台15南段_往南_v/c': ['11-12', 0.60429039117825],
  '台15南段_往北_v/c': ['08-09', 0.6824778163123131],
  '延平路_往南_v/c': ['08-09', 0.5200014227233255],
  '延平路_往北_v/c': ['08-09', 0.5980235156512274],
  '台68_往東_v/c': ['08-09', 0.19555355544099007],
  '台68_往西_v/c': ['09-10', 0.0390109021175971],
  '榮濱路_往南_v/c': ['09-10', 0.23683595778810312],
  '榮濱路_往北_v/c': ['08-09', 0.286353369426326],
  '南寮大道_往東_v/c': ['11-12', 0.029026007817539723],
  '南寮大道_往西_v/c': ['09-10', 0.041763766085231344]},
 {'海濱路_往北_v/c': ['06-07', 0.09647617507247398],
  '海濱路_往南_v/c': ['10-11', 0.16636213990228021],
  '天府路_往東_v/c': ['09-10', 0.03886979208673971],
  '天府路_往西_v/c': ['11-12', 0.07448990722680889],
  '台15北段_往南_v/c': ['10-11', 0.300056

In [40]:
rushhour_night_vc_wo, rushhour_night_vc_ho

({'海濱路_往北_v/c': ['17-18', 0.16217601229608128],
  '海濱路_往南_v/c': ['17-18', 0.23149479547774995],
  '天府路_往東_v/c': ['18-19', 0.03432181010581238],
  '天府路_往西_v/c': ['17-18', 0.09471019234466847],
  '台15北段_往南_v/c': ['18-19', 0.4263601482005171],
  '台15北段_往北_v/c': ['17-18', 0.4238995196026793],
  '台15南段_往南_v/c': ['17-18', 0.6000248119464036],
  '台15南段_往北_v/c': ['17-18', 0.43200543217489357],
  '延平路_往南_v/c': ['17-18', 0.5608935533834931],
  '延平路_往北_v/c': ['17-18', 0.4834573255916642],
  '台68_往東_v/c': ['17-18', 0.18962769012459643],
  '台68_往西_v/c': ['16-17', 0.06833282200990864],
  '榮濱路_往南_v/c': ['17-18', 0.29054696964361937],
  '榮濱路_往北_v/c': ['17-18', 0.21875382033887952],
  '南寮大道_往東_v/c': ['17-18', 0.06385721719858739],
  '南寮大道_往西_v/c': ['16-17', 0.07669346135651574]},
 {'海濱路_往北_v/c': ['17-18', 0.21487875357051023],
  '海濱路_往南_v/c': ['17-18', 0.1792173961674564],
  '天府路_往東_v/c': ['17-18', 0.11720737306155359],
  '天府路_往西_v/c': ['18-19', 0.13508845337618586],
  '台15北段_往南_v/c': ['17-18', 0.32771

In [41]:
columns_road_name = ['海濱路_往北', '海濱路_往南', '天府路_往東', '天府路_往西',
                   '台15北段_往南', '台15北段_往北', '台15南段_往南', '台15南段_往北', '延平路_往南', '延平路_往北',
                   '台68_往東', '台68_往西', '榮濱路_往南', '榮濱路_往北', '南寮大道_往東', '南寮大道_往西']

#index_item_name_平 = ['尖峰時間_平', 'PCU_平', 'c_平', 'v/c_平']
#index_item_name_假 = ['尖峰時間_假', 'PCU_假', 'c_假', 'v/c_假']

In [42]:
dicts_all = [rushhour_day_PCU_wo, rushhour_day_PCU_ho,
             rushhour_night_PCU_wo, rushhour_night_PCU_ho,
             rushhour_c_wo, rushhour_c_ho,
             rushhour_day_vc_wo, rushhour_day_vc_ho,
             rushhour_night_vc_wo, rushhour_night_vc_ho]

dicts_all_index = ['平日晨峰尖峰', '平日晨峰_PCU', '假日晨峰尖峰', '假日晨峰_PCU',
                   '平日昏峰尖峰', '平日昏峰_PCU', '假日昏峰尖峰', '假日昏峰_PCU',
                   '平日c值', '假日c值',
                   '平日晨峰尖峰', '平日晨峰_vc','假日晨峰尖峰', '假日晨峰_vc',
                   '平日昏峰尖峰', '平日昏峰_vc','假日昏峰尖峰', '假日昏峰_vc']
len(dicts_all_index)

18

In [43]:
list(rushhour_night_vc_wo)

['海濱路_往北_v/c',
 '海濱路_往南_v/c',
 '天府路_往東_v/c',
 '天府路_往西_v/c',
 '台15北段_往南_v/c',
 '台15北段_往北_v/c',
 '台15南段_往南_v/c',
 '台15南段_往北_v/c',
 '延平路_往南_v/c',
 '延平路_往北_v/c',
 '台68_往東_v/c',
 '台68_往西_v/c',
 '榮濱路_往南_v/c',
 '榮濱路_往北_v/c',
 '南寮大道_往東_v/c',
 '南寮大道_往西_v/c']

In [44]:
for dicts in dicts_all:
    for key, new_key in zip(list(dicts), columns_road_name):
        dicts[new_key] = dicts.pop(key)

In [45]:
rushhour_day_PCU_wo

{'海濱路_往北': ['08-09', 182.5],
 '海濱路_往南': ['08-09', 149.0],
 '天府路_往東': ['10-11', 77.0],
 '天府路_往西': ['09-10', 135.0],
 '台15北段_往南': ['11-12', 1294.5],
 '台15北段_往北': ['08-09', 1582.0],
 '台15南段_往南': ['11-12', 1487.5],
 '台15南段_往北': ['08-09', 1579.0],
 '延平路_往南': ['08-09', 426.0],
 '延平路_往北': ['08-09', 498.5],
 '台68_往東': ['08-09', 379.5],
 '台68_往西': ['09-10', 76.5],
 '榮濱路_往南': ['09-10', 280.0],
 '榮濱路_往北': ['08-09', 341.0],
 '南寮大道_往東': ['11-12', 55.0],
 '南寮大道_往西': ['09-10', 82.5]}

In [46]:
rushhour_night_PCU_wo

{'海濱路_往北': ['17-18', 214.0],
 '海濱路_往南': ['17-18', 140.5],
 '天府路_往東': ['18-19', 88.5],
 '天府路_往西': ['17-18', 237.5],
 '台15北段_往南': ['18-19', 1024.5],
 '台15北段_往北': ['17-18', 1003.0],
 '台15南段_往南': ['17-18', 1477.0],
 '台15南段_往北': ['17-18', 999.5],
 '延平路_往南': ['17-18', 459.5],
 '延平路_往北': ['17-18', 403.0],
 '台68_往東': ['17-18', 368.0],
 '台68_往西': ['16-17', 134.0],
 '榮濱路_往南': ['17-18', 343.5],
 '榮濱路_往北': ['17-18', 260.5],
 '南寮大道_往東': ['17-18', 121.0],
 '南寮大道_往西': ['16-17', 151.5]}

In [47]:
rushhour_day_PCU_ho

{'海濱路_往北': ['06-07', 121.0],
 '海濱路_往南': ['10-11', 110.0],
 '天府路_往東': ['09-10', 97.5],
 '天府路_往西': ['11-12', 185.0],
 '台15北段_往南': ['10-11', 748.5],
 '台15北段_往北': ['11-12', 574.5],
 '台15南段_往南': ['10-11', 942.0],
 '台15南段_往北': ['11-12', 572.5],
 '延平路_往南': ['11-12', 201.5],
 '延平路_往北': ['09-10', 194.0],
 '台68_往東': ['11-12', 318.5],
 '台68_往西': ['11-12', 223.0],
 '榮濱路_往南': ['11-12', 278.0],
 '榮濱路_往北': ['11-12', 201.5],
 '南寮大道_往東': ['11-12', 134.5],
 '南寮大道_往西': ['11-12', 240.5]}

In [48]:
rushhour_night_PCU_ho

{'海濱路_往北': ['17-18', 269.5],
 '海濱路_往南': ['17-18', 118.5],
 '天府路_往東': ['17-18', 294.0],
 '天府路_往西': ['18-19', 335.5],
 '台15北段_往南': ['17-18', 817.5],
 '台15北段_往北': ['17-18', 1315.0],
 '台15南段_往南': ['17-18', 1036.0],
 '台15南段_往北': ['17-18', 1309.5],
 '延平路_往南': ['16-17', 231.0],
 '延平路_往北': ['17-18', 290.0],
 '台68_往東': ['19-20', 789.5],
 '台68_往西': ['16-17', 544.0],
 '榮濱路_往南': ['16-17', 470.0],
 '榮濱路_往北': ['19-20', 337.0],
 '南寮大道_往東': ['19-20', 474.0],
 '南寮大道_往西': ['16-17', 607.5]}

In [49]:
rushhour_c_wo

{'海濱路_往北': [1319.5539646720674],
 '海濱路_往南': [606.9250918148789],
 '天府路_往東': [2578.5353315328953],
 '天府路_往西': [2507.649853942775],
 '台15北段_往南': [2402.8981233916306],
 '台15北段_往北': [2366.1267673530538],
 '台15南段_往南': [2461.564872973838],
 '台15南段_往北': [2313.6283147369345],
 '延平路_往南': [819.2285278162781],
 '延平路_往北': [833.5792606034069],
 '台68_往東': [1940.6448486410534],
 '台68_往西': [1960.9902834185486],
 '榮濱路_往南': [1182.2529087855642],
 '榮濱路_往北': [1190.8363456073587],
 '南寮大道_往東': [1894.8523801735082],
 '南寮大道_往西': [1975.3965634141878]}

In [50]:
rushhour_c_ho

{'海濱路_往北': [1254.1956592817185],
 '海濱路_往南': [661.2081334407764],
 '天府路_往東': [2508.3746211563034],
 '天府路_往西': [2483.55793271036],
 '台15北段_往南': [2494.5299100606435],
 '台15北段_往北': [2468.150169441981],
 '台15南段_往南': [2523.542223028908],
 '台15南段_往北': [2407.3694142449776],
 '延平路_往南': [833.5144223484758],
 '延平路_往北': [814.1126155688734],
 '台68_往東': [1939.329204098458],
 '台68_往西': [1935.010417727184],
 '榮濱路_往南': [1206.9523199770147],
 '榮濱路_往北': [1190.7242615056214],
 '南寮大道_往東': [1881.700115280525],
 '南寮大道_往西': [1933.1528518408768]}

In [51]:
rushhour_day_vc_wo

{'海濱路_往北': ['08-09', 0.13830430955156464],
 '海濱路_往南': ['08-09', 0.24549981869170634],
 '天府路_往東': ['10-11', 0.029861913877373483],
 '天府路_往西': ['09-10', 0.053835267227495764],
 '台15北段_往南': ['11-12', 0.5387244625139769],
 '台15北段_往北': ['08-09', 0.6686032303204772],
 '台15南段_往南': ['11-12', 0.60429039117825],
 '台15南段_往北': ['08-09', 0.6824778163123131],
 '延平路_往南': ['08-09', 0.5200014227233255],
 '延平路_往北': ['08-09', 0.5980235156512274],
 '台68_往東': ['08-09', 0.19555355544099007],
 '台68_往西': ['09-10', 0.0390109021175971],
 '榮濱路_往南': ['09-10', 0.23683595778810312],
 '榮濱路_往北': ['08-09', 0.286353369426326],
 '南寮大道_往東': ['11-12', 0.029026007817539723],
 '南寮大道_往西': ['09-10', 0.041763766085231344]}

In [52]:
rushhour_night_vc_wo

{'海濱路_往北': ['17-18', 0.16217601229608128],
 '海濱路_往南': ['17-18', 0.23149479547774995],
 '天府路_往東': ['18-19', 0.03432181010581238],
 '天府路_往西': ['17-18', 0.09471019234466847],
 '台15北段_往南': ['18-19', 0.4263601482005171],
 '台15北段_往北': ['17-18', 0.4238995196026793],
 '台15南段_往南': ['17-18', 0.6000248119464036],
 '台15南段_往北': ['17-18', 0.43200543217489357],
 '延平路_往南': ['17-18', 0.5608935533834931],
 '延平路_往北': ['17-18', 0.4834573255916642],
 '台68_往東': ['17-18', 0.18962769012459643],
 '台68_往西': ['16-17', 0.06833282200990864],
 '榮濱路_往南': ['17-18', 0.29054696964361937],
 '榮濱路_往北': ['17-18', 0.21875382033887952],
 '南寮大道_往東': ['17-18', 0.06385721719858739],
 '南寮大道_往西': ['16-17', 0.07669346135651574]}

In [53]:
rushhour_day_vc_ho

{'海濱路_往北': ['06-07', 0.09647617507247398],
 '海濱路_往南': ['10-11', 0.16636213990228021],
 '天府路_往東': ['09-10', 0.03886979208673971],
 '天府路_往西': ['11-12', 0.07448990722680889],
 '台15北段_往南': ['10-11', 0.30005653449222564],
 '台15北段_往北': ['11-12', 0.23276541561889144],
 '台15南段_往南': ['10-11', 0.37328481822244075],
 '台15南段_往北': ['11-12', 0.23781144539445476],
 '延平路_往南': ['11-12', 0.24174746662722635],
 '延平路_往北': ['09-10', 0.23829627042990803],
 '台68_往東': ['11-12', 0.1642320444238667],
 '台68_往西': ['11-12', 0.11524485757649323],
 '榮濱路_往南': ['11-12', 0.23033221395630132],
 '榮濱路_往北': ['11-12', 0.16922473700604002],
 '南寮大道_往東': ['11-12', 0.07147791452409442],
 '南寮大道_往西': ['11-12', 0.1244081655369258]}

In [54]:
rushhour_night_vc_ho

{'海濱路_往北': ['17-18', 0.21487875357051023],
 '海濱路_往南': ['17-18', 0.1792173961674564],
 '天府路_往東': ['17-18', 0.11720737306155359],
 '天府路_往西': ['18-19', 0.13508845337618586],
 '台15北段_往南': ['17-18', 0.32771705670994583],
 '台15北段_往北': ['17-18', 0.5327876789187854],
 '台15南段_往南': ['17-18', 0.41053404636777985],
 '台15南段_往北': ['17-18', 0.5439547384175345],
 '延平路_往南': ['16-17', 0.2771397756371677],
 '延平路_往北': ['17-18', 0.3562160743539862],
 '台68_往東': ['19-20', 0.4070995261307465],
 '台68_往西': ['16-17', 0.2811354373166472],
 '榮濱路_往南': ['16-17', 0.38941057755202024],
 '榮濱路_往北': ['19-20', 0.28302102417387337],
 '南寮大道_往東': ['19-20', 0.2518998623377008],
 '南寮大道_往西': ['16-17', 0.3142534742772658]}

In [55]:
frame = []
for dicts in dicts_all:
    df = pd.DataFrame(dicts)
    frame.append(df)

In [56]:
df_combine = pd.concat(frame)

In [57]:
df_combine_trans = df_combine.T

In [58]:
df_combine_trans.columns = dicts_all_index

In [59]:
df_combine_trans

,平日晨峰尖峰,平日晨峰_PCU,假日晨峰尖峰,假日晨峰_PCU,平日昏峰尖峰,平日昏峰_PCU,假日昏峰尖峰,假日昏峰_PCU,平日c值,假日c值,平日晨峰尖峰,平日晨峰_vc,假日晨峰尖峰,假日晨峰_vc,平日昏峰尖峰,平日昏峰_vc,假日昏峰尖峰,假日昏峰_vc
海濱路_往北,08-09,182.5,06-07,121,17-18,214,17-18,269.5,1319.55,1254.2,08-09,0.138304,06-07,0.0964762,17-18,0.162176,17-18,0.214879
海濱路_往南,08-09,149,10-11,110,17-18,140.5,17-18,118.5,606.925,661.208,08-09,0.2455,10-11,0.166362,17-18,0.231495,17-18,0.179217
天府路_往東,10-11,77,09-10,97.5,18-19,88.5,17-18,294,2578.54,2508.37,10-11,0.0298619,09-10,0.0388698,18-19,0.0343218,17-18,0.117207
天府路_往西,09-10,135,11-12,185,17-18,237.5,18-19,335.5,2507.65,2483.56,09-10,0.0538353,11-12,0.0744899,17-18,0.0947102,18-19,0.135088
台15北段_往南,11-12,1294.5,10-11,748.5,18-19,1024.5,17-18,817.5,2402.9,2494.53,11-12,0.538724,10-11,0.300057,18-19,0.42636,17-18,0.327717
台15北段_往北,08-09,1582,11-12,574.5,17-18,1003,17-18,1315,2366.13,2468.15,08-09,0.668603,11-12,0.232765,17-18,0.4239,17-18,0.532788
台15南段_往南,11-12,1487.5,10-11,942,17-18,1477,17-18,1036,2461.56,2523.54,11-12,0.60429,10-11,0.373285,17-18,0.600025,17-18,0.410534
台15南段_往北,08-09,1579,11-12,572.5,17-18,999.5,17-18,1309.5,2313.63,2407.37,08-09,0.682478,11-12,0.237811,17-18,0.432005,17-18,0.543955
延平路_往南,08-09,426,11-12,201.5,17-18,459.5,16-17,231,819.229,833.514,08-09,0.520001,11-12,0.241747,17-18,0.560894,16-17,0.27714
延平路_往北,08-09,498.5,09-10,194,17-18,403,17-18,290,833.579,814.113,08-09,0.598024,09-10,0.238296,17-18,0.483457,17-18,0.356216


In [60]:
writer = pd.ExcelWriter('新竹海淡_交通量整理(方向)_2nd.xlsx')
df1_woday.to_excel(writer, sheet_name='天府路及海濱路路口(平日)')
df1_hoday.to_excel(writer, sheet_name='天府路及海濱路路口(假日)')
df2_woday.to_excel(writer, sheet_name='台15線及延平路口(平日)')
df2_hoday.to_excel(writer, sheet_name='台15線及延平路口(假日)')
df3_woday.to_excel(writer,sheet_name='南寮大道及榮濱路(平日)')
df3_hoday.to_excel(writer,sheet_name='南寮大道及榮濱路(假日)')

df_combine_trans.to_excel(writer, sheet_name='各路段尖峰時間統計')

writer.save()
writer.close()